In [1]:
# Import Statements
import eeg_utils
import numpy as np
from eeg_utils import *
from tensorflow import keras
from sklearn.model_selection import train_test_split

In [2]:
# Load EEG Data
X_test = np.load("X_test.npy")
y_test = np.load("y_test.npy")
person_train_valid = np.load("person_train_valid.npy")
X_train_valid = np.load("X_train_valid.npy")
y_train_valid = np.load("y_train_valid.npy")
person_test = np.load("person_test.npy")

In [3]:
# Visualize EEG Data Shapes
print("Training/Valid data shape: {}".format(X_train_valid.shape))
print("Test data shape: {}".format(X_test.shape))
print("Training/Valid target shape: {}".format(y_train_valid.shape))
print("Test target shape: {}".format(y_test.shape))
print("Person train/valid shape: {}".format(person_train_valid.shape))
print("Person test shape: {}".format(person_test.shape))

Training/Valid data shape: (2115, 22, 1000)
Test data shape: (443, 22, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115, 1)
Person test shape: (443, 1)


In [4]:
# [RUN IFF] Trained on Subject One ONLY
patient_data = []
patient_truths = []

test_data = []
test_truths = []

for patient in np.unique(person_train_valid):
    idx = np.where(person_train_valid == patient)[0]
    patient_data.append(X_train_valid[idx, :, :])
    patient_truths.append(y_train_valid[idx])
    pass

# ALso split the testing set, as we want to optimize on subject one
for patient in np.unique(person_test):
    idx = np.where(person_test == patient)[0]
    test_data.append(X_test[idx, :, :])
    test_truths.append(y_test[idx])
    pass

# Store actual test data
X_test_actual = X_test
y_test_actual = y_test

X_train_valid = patient_data[0]
y_train_valid = patient_truths[0]
X_test = test_data[0]
y_test = test_truths[0]

In [5]:
# Preprocess and Split Training Data
data_config = {"Duplicate": 2, "Mu": 0.0, "Sigma": 0.5, "Time_Point": 500}
X_train_valid, y_train_valid, X_test, y_test = vae_preprocess_data(X_train_valid,
                                                               y_train_valid,
                                                               X_test,
                                                               y_test, 
                                                               data_config)

X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid,
                                                      y_train_valid,
                                                      test_size=0.2,
                                                      random_state=0)

In [6]:
# Load Optimal Model
model_name = "./VAE_Train_One"
optim_model = keras.models.load_model(model_name) # Model that was trained on all subjects

In [7]:
# Evaluate Results (Train on one subject, Test on one subject)
optim_model.evaluate(X_train, y_train) # 54.09%
optim_model.evaluate(X_valid, y_valid) # 51.58%
optim_model.evaluate(X_test, y_test) # 46.00%

2/2 [==============================] - 0s 121ms/step - loss: 1.4330 - accuracy: 0.4600


[1.4330135583877563, 0.46000000834465027]

In [8]:
'''
Model Parameters: 6 Epochs
    - Data augmented VAE-CNN
    - Regularization : 0.006
    - Learning Rate : 0.001
    - Time Points : 500
    - Kernel_dim : (10, 1)
    - Channel_dim : 22
    - Dropout : 0.5
    - Duplicate : 2
    - Sigma : 0.5
    - Mu : 0.0
    
    N.B. For importance of the parameters above, see VAE_CNN Model notebook for architecture
'''

'\nModel Parameters: 6 Epochs\n    - Data augmented VAE-CNN\n    - Regularization : 0.006\n    - Learning Rate : 0.001\n    - Time Points : 500\n    - Kernel_dim : (10, 1)\n    - Channel_dim : 22\n    - Dropout : 0.5\n    - Duplicate : 2\n    - Sigma : 0.5\n    - Mu : 0.0\n    \n    N.B. For importance of the parameters above, see VAE_CNN Model notebook for architecture\n'